In [ ]:
# --- 2. Install Packages ---
!pip install -q \
    catboost \
    lightgbm \
    pytorch-tabnet \
    torch \
    scikit-learn-intelex \
    tqdm \
    pandas \
    openpyxl \
    xgboost \
    joblib

print("All packages installed! Runtime will restart automatically if needed.\n")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.3/111.3 MB 7.7 MB/s eta 0:00:00
All packages installed! Runtime will restart automatically if needed.



In [ ]:
# ==============================================================
# train_model.py – FULL VERSION (Google Drive + Fixed Stacking)
# ==============================================================

# --- 1. Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- 3. Imports & Safe Optional Models ---
import pandas as pd
import numpy as np
import joblib
import os
import warnings
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor,
    StackingRegressor
)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from tqdm import tqdm

# Safe imports
models_to_try = {}
try:
    from xgboost import XGBRegressor
    models_to_try['xgb'] = XGBRegressor
    print("XGBoost imported")
except: print("XGBoost not available")

try:
    from catboost import CatBoostRegressor
    models_to_try['cat'] = CatBoostRegressor
    print("CatBoost imported")
except: print("CatBoost not available")

try:
    from lightgbm import LGBMRegressor
    models_to_try['lgb'] = LGBMRegressor
    print("LightGBM imported")
except: print("LightGBM not available")

TABNET_AVAILABLE = False
try:
    from pytorch_tabnet.tab_model import TabNetRegressor
    import torch
    TABNET_AVAILABLE = True
    print("TabNet + PyTorch imported")
except: print("TabNet not available")

warnings.filterwarnings("ignore")

# --- 4. Google Drive Paths ---
DRIVE_ROOT = "/content/drive/MyDrive/ML_Sessile_Drop"
os.makedirs(DRIVE_ROOT, exist_ok=True)

CHECKPOINT_DIR = os.path.join(DRIVE_ROOT, "checkpoints")
MODEL_DIR = os.path.join(CHECKPOINT_DIR, "models")
os.makedirs(MODEL_DIR, exist_ok=True)

LOG_FILE = os.path.join(DRIVE_ROOT, "training_log.csv")
PREDICTIONS_FILE = os.path.join(DRIVE_ROOT, "test_predictions.csv")

# Initialize log
log_df = pd.DataFrame(columns=['timestamp','phase','model','param','mse','pct_error','note'])
if os.path.exists(LOG_FILE):
    log_df = pd.read_csv(LOG_FILE)

def _fmt(v, fmt="{:.2e}"):
    return fmt.format(v) if v is not None and not np.isnan(v) else "—"

def log_step(phase, model="", param="", mse=None, pct_error=None, note=""):
    global log_df
    ts = datetime.now().strftime("%H:%M:%S")
    row = pd.DataFrame([{
        'timestamp': ts, 'phase': phase, 'model': model, 'param': param,
        'mse': mse, 'pct_error': pct_error, 'note': note
    }])
    log_df = pd.concat([log_df, row], ignore_index=True)
    log_df.to_csv(LOG_FILE, index=False)
    print(f"[{ts}] {phase} | {model} | {note} | MSE:{_fmt(mse)} | %Err:{_fmt(pct_error,'{:.2f}%')}")

# --- 5. Load Data ---
print("\nLoading data...")
DATA_PATH = "/content/drive/MyDrive/Main_final.xlsx"  # CHANGE IF NEEDED
data = pd.read_excel(DATA_PATH)
data = data.drop_duplicates().copy()
data['Diameter_m'] = data['Diameter'] / 1000.0
data['Bo'] = 1000.0 * 9.81 * (data['Diameter_m']**2) / 0.072
log_step("load", note=f"{len(data)} rows, Bo ∈ [{data['Bo'].min():.3f},{data['Bo'].max():.3f}]")

# --- 6. Features & Target ---
feat_cols = ['Bo', 'ContactAngle', 'x']
X_raw = data[feat_cols]
y = data['y']

# --- 7. Preprocessing ---
print("Applying polynomial features (degree=2)...")
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_raw)
poly_names = poly.get_feature_names_out(feat_cols)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_poly)

X_train, X_test, y_train, y_test, idx_tr, idx_te = train_test_split(
    X_scaled, y, data.index, test_size=0.2, random_state=42)

print(f"Train: {len(X_train)} | Test: {len(X_test)}")

# Save preprocessing
joblib.dump(scaler, os.path.join(DRIVE_ROOT, "scaler.joblib"))
joblib.dump(poly, os.path.join(DRIVE_ROOT, "poly.joblib"))
joblib.dump(feat_cols, os.path.join(DRIVE_ROOT, "feature_cols.joblib"))

# --- 8. Safe % Error ---
def pct_err(y_true, y_pred):
    mask = y_true > 1e-6
    if not mask.any(): return np.inf
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# --- 9. Train Models ---
trained = {}

# 1. Random Forest
print("\nTraining Random Forest...")
rf = RandomForestRegressor(n_estimators=300, max_features='sqrt', random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
trained['rf'] = rf
joblib.dump(rf, os.path.join(MODEL_DIR, "rf_model.joblib"))
log_step("fit", "RF", note="fitted")

# 2. Gradient Boosting
print("\nTraining Gradient Boosting...")
gb = GradientBoostingRegressor(n_estimators=400, max_depth=5, learning_rate=0.05, random_state=42)
gb.fit(X_train, y_train)
trained['gb'] = gb
joblib.dump(gb, os.path.join(MODEL_DIR, "gb_model.joblib"))
log_step("fit", "GB", note="fitted")

# 3. HistGradientBoosting
print("\nTraining HistGradientBoosting...")
hgb = HistGradientBoostingRegressor(max_iter=600, max_depth=7, learning_rate=0.05, random_state=42)
hgb.fit(X_train, y_train)
trained['hgb'] = hgb
joblib.dump(hgb, os.path.join(MODEL_DIR, "hgb_model.joblib"))
log_step("fit", "HGB", note="fitted")

# 4. XGBoost
if 'xgb' in models_to_try:
    print("\nTraining XGBoost...")
    xgb = XGBRegressor(n_estimators=500, max_depth=6, learning_rate=0.05, n_jobs=-1, random_state=42)
    xgb.fit(X_train, y_train)
    trained['xgb'] = xgb
    joblib.dump(xgb, os.path.join(MODEL_DIR, "xgb_model.joblib"))
    log_step("fit", "XGB", note="fitted")

# 5. CatBoost
if 'cat' in models_to_try:
    print("\nTraining CatBoost...")
    cat = CatBoostRegressor(iterations=600, depth=6, learning_rate=0.05, verbose=0, random_seed=42)
    cat.fit(X_train, y_train)
    trained['cat'] = cat
    joblib.dump(cat, os.path.join(MODEL_DIR, "cat_model.joblib"))
    log_step("fit", "CAT", note="fitted")

# 6. LightGBM
if 'lgb' in models_to_try:
    print("\nTraining LightGBM...")
    lgb = LGBMRegressor(n_estimators=600, max_depth=7, learning_rate=0.05, n_jobs=-1, random_state=42)
    lgb.fit(X_train, y_train)
    trained['lgb'] = lgb
    joblib.dump(lgb, os.path.join(MODEL_DIR, "lgb_model.joblib"))
    log_step("fit", "LGB", note="fitted")

# 7. Neural Network
print("\nTraining Neural Network...")
nn = MLPRegressor(hidden_layer_sizes=(128,64,32), max_iter=2000, early_stopping=True,
                  validation_fraction=0.1, learning_rate='adaptive', random_state=42)
nn.fit(X_train, y_train)
trained['nn'] = nn
joblib.dump(nn, os.path.join(MODEL_DIR, "nn_model.joblib"))
log_step("fit", "NN", note="fitted")

# 8. TabNet (standalone, not in stacking)
if TABNET_AVAILABLE:
    print("\nTraining TabNet (standalone)...")
    tabnet = TabNetRegressor(n_d=64, n_a=64, n_steps=5, gamma=1.5, lambda_sparse=1e-4,
                             optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=2e-2), verbose=0)
    tabnet.fit(X_train, y_train.values.reshape(-1,1), max_epochs=200, patience=50, batch_size=256)
    trained['tabnet'] = tabnet
    joblib.dump(tabnet, os.path.join(MODEL_DIR, "tabnet_model.joblib"))
    log_step("fit", "TABNET", note="fitted (standalone)")

# --- 10. Stacking (ONLY sklearn-compatible models) ---
print("\nTraining Stacking Ensemble (sklearn models only)...")
valid_base_models = []
for name, model in trained.items():
    if name in ['tabnet', 'stack']:
        continue
    if hasattr(model, 'fit') and hasattr(model, 'predict'):
        valid_base_models.append((name, model))

if len(valid_base_models) < 2:
    print("Not enough valid models for stacking. Skipping.")
    log_step("fit", "STACK", note="skipped")
else:
    stack = StackingRegressor(
        estimators=valid_base_models,
        final_estimator=LinearRegression(),
        cv=5,
        n_jobs=-1
    )
    stack.fit(X_train, y_train)
    trained['stack'] = stack
    joblib.dump(stack, os.path.join(MODEL_DIR, "stack_model.joblib"))
    names = ', '.join([n for n,_ in valid_base_models])
    log_step("fit", "STACK", note=f"{len(valid_base_models)} models: {names}")

# --- 11. Evaluation ---
print("\nEvaluating on test set...")
test_df = data.loc[idx_te, ['Bo','ContactAngle','x','y']].copy().reset_index(drop=True)
test_df.rename(columns={'y':'y_true'}, inplace=True)

results = {}
for name, model in trained.items():
    if name == 'tabnet':
        pred = model.predict(X_test).ravel()
    else:
        pred = model.predict(X_test)
    mse = mean_squared_error(y_test, pred)
    err = pct_err(y_test.values, pred)
    results[name] = {'mse':mse, 'pct_error':err}
    test_df[f'y_pred_{name}'] = pred
    log_step("eval", name.upper(), mse=mse, pct_error=err, note="TEST")

# Save predictions
test_df.to_csv(PREDICTIONS_FILE, index=False)

# Save feature importance
for n in ['rf','gb','hgb','xgb','cat','lgb']:
    if n in trained and hasattr(trained[n], 'feature_importances_'):
        pd.Series(trained[n].feature_importances_, index=poly_names)\
          .to_csv(os.path.join(DRIVE_ROOT, f"importance_{n}.csv"))

print(f"\nSAVED TO GOOGLE DRIVE:")
print(f"   Folder: {DRIVE_ROOT}")
print(f"   Models: {MODEL_DIR}")
print(f"   Predictions: {PREDICTIONS_FILE}")
print(f"   Log: {LOG_FILE}")

# --- 12. Final Ranking ---
print("\n=== FINAL RANKING ===")
rank = sorted(results.items(), key=lambda x: x[1]['pct_error'])
for n, r in rank:
    print(f"   {n.upper():8} -> %Err: {r['pct_error']:.3f}% | MSE: {r['mse']:.2e}")
print(f"\nBEST MODEL: {rank[0][0].upper()} ({rank[0][1]['pct_error']:.3f}%)")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
All packages installed! Runtime will restart automatically if needed.

XGBoost imported
CatBoost imported
LightGBM imported
TabNet + PyTorch imported

Loading data...
[10:14:21] load |  | 58976 rows, Bo ∈ [0.085,5.494] | MSE:— | %Err:—
Applying polynomial features (degree=2)...
Train: 47180 | Test: 11796

Training Random Forest...
[10:14:53] fit | RF | fitted | MSE:— | %Err:—

Training Gradient Boosting...
[10:16:07] fit | GB | fitted | MSE:— | %Err:—

Training HistGradientBoosting...
[10:16:07] fit | HGB | fitted | MSE:— | %Err:—

Training XGBoost...
[10:16:08] fit | XGB | fitted | MSE:— | %Err:—

Training CatBoost...
[10:16:11] fit | CAT | fitted | MSE:— | %Err:—

Training LightGBM...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001011 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory

In [ ]:
# ==============================================================
# predict_model.py – SMART PLOT CHOOSER (DNS may be missing!)
# ==============================================================

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd, numpy as np, matplotlib.pyplot as plt, joblib, os, logging
from scipy.interpolate import interp1d
from scipy.spatial import cKDTree
from datetime import datetime

# ─── PATHS & LOG ───
DRIVE_ROOT = "/content/drive/MyDrive/ML_Sessile_Drop"
LOG_FILE   = os.path.join(DRIVE_ROOT, "prediction_log.txt")
logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(message)s',
                    filename=LOG_FILE, filemode='a')
logging.info("=== SMART PREDICTION START ===")

MODEL_DIR  = os.path.join(DRIVE_ROOT, "checkpoints", "models")
DNS_PATH   = "/content/drive/MyDrive/Main_final.xlsx"
OUT        = DRIVE_ROOT
os.makedirs(OUT, exist_ok=True)

Δρ, g, σ = 1000.0, 9.81, 0.072

# ─── LOAD DNS ONCE ───
df = pd.read_excel(DNS_PATH).drop_duplicates()
df['Bo'] = Δρ * g * (df['Diameter']/1000)**2 / σ
print(f"Loaded {len(df):,} DNS points")

# ─── LIST MODELS ───
models = {f.replace(".joblib","").replace("_model",""): os.path.join(MODEL_DIR,f)
          for f in os.listdir(MODEL_DIR) if f.endswith(".joblib")}
for i,k in enumerate(models,1): print(f" [{i}] {k}")
choice = input("\nPick model → ").strip().lower()
if choice not in models: raise ValueError("Model not found!")
model = joblib.load(models[choice])
scaler, poly, feat_cols = [joblib.load(os.path.join(DRIVE_ROOT,f))
                           for f in ["scaler.joblib","poly.joblib","feature_cols.joblib"]]
logging.info(f"Model: {choice}")

# ─── USER INPUT ───
D = float(input("\nDiameter (mm) → "))
θ = float(input("Contact angle (°) → "))
if D<=0 or not(0<=θ<=180): raise ValueError("Bad input!")

D_scaled = round(D * 2**(-1/3), 2)
Bo = Δρ * g * (D_scaled/1000)**2 / σ
print(f"\nScaled D = {D_scaled} mm → Bo = {Bo:.6f}")

# ─── FIND NEAREST DNS (or None) ───
tree = cKDTree(df[['Bo','ContactAngle']].drop_duplicates())
dist, idx = tree.query([Bo, θ], k=1)
nearBo, nearCA = df[['Bo','ContactAngle']].drop_duplicates().iloc[idx].values
mask = (abs(df['Bo']-nearBo)<1e-9) & (abs(df['ContactAngle']-nearCA)<1e-6)
dns_profile = df[mask].sort_values('x')

if len(dns_profile)==0:
    print("DNS NOT FOUND → Will still plot ML drop!")
    dns_ok = False
    x_dns, y_dns = np.array([]), np.array([])
else:
    dns_ok = True
    x_dns, y_dns = dns_profile['x'].values, dns_profile['y'].values
    print(f"Nearest DNS: Bo={nearBo:.5f}, θ={nearCA}° (dist={dist:.3f})")

logging.info(f"D={D}, Dsc={D_scaled}, Bo={Bo:.6f}, CA={θ}, DNS={'YES' if dns_ok else 'NO'}")

# ─── ML PREDICTION ───
input_df = pd.DataFrame({c: [Bo if c=='Bo' else θ if c=='ContactAngle' else x_dns][0]
                         for c in feat_cols})
if len(x_dns)==0:  # fallback grid
    x_dns = np.linspace(-0.6, 0.6, 200) * (D_scaled/2)
input_df = pd.DataFrame({
    feat_cols[0]: [Bo] * len(x_dns),
    feat_cols[1]: [θ] * len(x_dns),
    feat_cols[2]: x_dns
})
X = scaler.transform(poly.transform(input_df))
y_ml = model.predict(X).ravel() if choice!='tabnet' else model.predict(X).ravel()

# ─── CLEAN REPEATS ───
def clean(x,y,max_rep=3):
    fx,fy,c,p=[],[],0,None
    for xi,yi in zip(x,y):
        if yi==p: c+=1
        else: c=0
        if c<=max_rep: fx.append(xi); fy.append(yi)
        p=yi
    return np.array(fx),np.array(fy)
x_ml,y_ml = clean(x_dns,y_ml)
x_dns,y_dns = clean(x_dns,y_dns)

# ─── MIRROR ───
mirror = lambda x,y: (np.concatenate([x,x[::-1]]),
                      np.concatenate([y,-y[::-1]]))

# ─── SAVE EXCEL ───
def save_xl(name,x,y):
    pd.DataFrame({'x_mm':x*1000,'y_mm':y*1000}).to_excel(
        f"{OUT}/{name}_D{D_scaled}_θ{θ}.xlsx", index=False)
    print(f"Excel → {name}")

save_xl(f"ML_{choice}", x_ml, y_ml)
if dns_ok: save_xl("DNS_nearest", x_dns, y_dns)

# ─── PLOT ENGINE ───
def plot(ax, x, y, label, col):
    xu,yu = mirror(x,y)
    ax.plot(xu*1000, yu*1000, 'o-', color=col, mfc='none', ms=3, label=label)

def finish(fig,ax,title,fname):
    ax.axhline(0,color='k',lw=2)
    ax.set_xlabel('x (mm)'); ax.set_ylabel('y (mm)')
    ax.set_title(title, fontsize=13)
    ax.grid(True,ls='--',alpha=0.4)
    ax.axis('equal'); ax.legend()
    fig.savefig(f"{OUT}/{fname}", dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"Plot → {fname}")

# ─── SMART MENU ───
print("\nWHAT DO YOU WANT TO SEE?")
print("  1 → ML only")
print("  2 → DNS only" + ("" if dns_ok else " (NOT AVAILABLE)"))
print("  3 → Both together")
opt = input("Choose 1 / 2 / 3 → ").strip()

if opt=="1" or not dns_ok:                     # ALWAYS WORKS
    fig,ax=plt.subplots(figsize=(8,6))
    plot(ax, x_ml, y_ml, choice.upper(), 'red')
    finish(fig,ax, f"{choice.upper()} – D={D_scaled} mm, θ={θ}°",
           f"ML_{choice}_D{D_scaled}_θ{θ}.png")

if opt=="2" and dns_ok:
    fig,ax=plt.subplots(figsize=(8,6))
    plot(ax, x_dns, y_dns, 'DNS', 'blue')
    finish(fig,ax, f"DNS (Bo={nearBo:.3f}, θ={nearCA}°)", f"DNS_D{D_scaled}_θ{θ}.png")

if opt=="3" and dns_ok:
    fig,ax=plt.subplots(figsize=(9,6))
    plot(ax, x_ml, y_ml, choice.upper(), 'red')
    plot(ax, x_dns, y_dns, 'DNS', 'blue')
    finish(fig,ax, f"Comparison – D={D_scaled} mm, θ={θ}°",
           f"COMPARE_D{D_scaled}_θ{θ}.png")

# ─── % ERROR (only if DNS exists) ───
if dns_ok:
    interp = interp1d(x_dns, y_dns, kind='linear', fill_value='extrapolate')
    y_dns_i = interp(x_ml)
    m = (y_dns_i>0) & (y_ml>0)
    if m.sum():
        err = np.mean(abs((y_dns_i[m]-y_ml[m])/y_dns_i[m]))*100
        print(f"\n% Error = {err:.3f}%")
        logging.info(f"%Error={err:.3f}%")

logging.info("=== END ===")
print(f"\nEVERYTHING SAVED → {OUT}")